# Prima prova poco efficiente ma carina

---

## Fase preliminare

#### Importazioni e definizioni

In [ ]:
# Importo
import numpy as np
# Definisco
nome_file = '../../data/fcc100a108.txt' # nome file da leggere
R_C = 4.5 # raggio di cutoff (angstrom)

#### Lettura file

In [3]:
# Leggo il file, raccolgo le coordinate in tre vettori
data = np.loadtxt(nome_file)
vec_x = data[:, 0] 
vec_y = data[:, 1]  
vec_z = data[:, 2]   
# Definisco due liste
n_neighbours = [] # ogni entrata è il numero di primi vicini dell'i-esimo atomo
which_neighbour = [] # ogni entrata è la lista di indici dei primi vicini dell'i-esimo atomo
how_distant = []


#### Calcolo distanza e controllo vicini

In [4]:
for i, (x_i, y_i, z_i) in enumerate(zip(vec_x, vec_y, vec_z)):
    n_neigh_i = 0  # contatore dei primi vicini dell'i-esimo atomo
    list_neigh_i = [] # lista di indici dei primi vicini dell'i-esimo atomo
    list_dist_ij = [] # lista di distanze tra i primi vicini 
    
    for j, (x_j, y_j, z_j) in enumerate(zip(vec_x, vec_y, vec_z)):
        if i != j:  # non considero la distanza di un atomo da se stesso
            dx = (x_i - x_j)**2 
            dy = (y_i - y_j)**2
            dz = (z_i - z_j)**2
            d_ij = np.sqrt(dx + dy + dz)
            
            if d_ij <= R_C:
                n_neigh_i += 1
                list_neigh_i.append(j)
                list_dist_ij.append(d_ij)
    
    # Aggiungo i risultati per l'atomo i-esimo
    n_neighbours.append(n_neigh_i)
    which_neighbour.append(list_neigh_i)
    how_distant.append(list_dist_ij)

___

## Calcolo del potenziale

Calcolo il potenziale $V$:
$$
V = \frac12 \sum_{i\neq j,\ i,j=1}^{N} \phi(r_{ij})
$$
con $\phi$ potenziale di Lennard-Jonnes:
$$
\phi(r_{ij}) = 4\varepsilon\left[
    \left( \frac{\sigma}{r_{ij}} \right)^{12} -
    \left( \frac{\sigma}{r_{ij}} \right)^{6}
    \right]
$$
e, per l'argento (Ag), i parametri sono: $\varepsilon=0.345\ \text{eV}$ e $\sigma=2.644\ \text{\AA}$.

Scritto così però il costo per calcolare $V$ scala come $\mathcal{O}(N^2)$ e non va bene. Abbiamo già i primi vicini per ogni atomo $i$, quindi calcolo $V_{ij}$ **soltanto** su quelli.

#### Lennard-Jones

In [5]:
def lennard_jones_ij(r_ij, sigma=2.644, epsilon=0.345):
    twelve = (sigma/r_ij)**12
    six = (sigma/r_ij)**6
    return 4*epsilon*(twelve - six)

#### Calcolo

L'ho scritto in una maniera che costicchia un po' di più sulla RAM, siccome tengo in memoria i valori di distanza; potrei ricalcolarli, ma a quel punto verrebbe più costoso sulla CPU. 

Il notebook è tanto carino per spiegare cosa sto facendo, ma conviene farlo in un .py così mi evito di ciclare mille volte sugli stessi indici!

In [6]:
potenziale = 0
for i, atom_neighbours in enumerate(which_neighbour):
    for j, neighbour_index in enumerate(atom_neighbours):
        if i != neighbour_index:
            d_ij = how_distant[i][j]
            potenziale += lennard_jones_ij(d_ij)
potenziale = potenziale/2

Salvo in un file

---

## Forza sul $k$-esimo atomo

È la derivata del potenziale rispetto alle coordinate del $k$-esimo atomo
$$
\vec{F}_k = -\vec{\nabla}_k\ V = 
-\left(
\frac{\partial V}{\partial x_k}, 
\frac{\partial V}{\partial y_k}, 
\frac{\partial V}{\partial z_k}
\right)
$$
per esempio, sulla coordinata $x$ sarà:
$$
{F_k}_x = -\frac12\frac{\partial}{\partial x_k}\sum_{i\neq j} \phi(r_{ij}) =
-\frac12 \sum_{i\neq j} \frac{\partial\phi(r_{ij})}{\partial x_k}
$$
qui si usa la *chain rule*
$$
{F_k}_x = -\frac12\sum_{i\neq j}\frac{\partial\phi(r_{ij})}{\partial x_k} = 
-\frac12\sum_{i\neq j}\frac{\partial r_{ij}}{\partial x_k}
\frac{\partial\phi(r_{ij})}{\partial r_{ij}}
$$
dove:
$$
\begin{align*}
\frac{\partial r_{ij}}{\partial x_k} &= \frac{\partial}{\partial x_k}
\left[ (x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2 \right]^{\frac12}=\\ 
&= 2(x_i-x_j)\frac{1}{2}\left[ (x_i-x_j)^2+(y_i-y_j)^2+(z_i-z_j)^2 \right]^{-\frac12}\delta_{ik}=\\
&= \frac{(x_i-x_j)}{r_{ij}}\delta_{ik}
\end{align*}
$$
e: 
$$
\begin{align*}
\frac{\partial\phi(r_{ij})}{\partial r_{ij}} &= \frac{\partial}{\partial r_{ij}}
4\varepsilon\left[
\left( \frac{\sigma}{r_{ij}} \right)^{12} -
\left( \frac{\sigma}{r_{ij}} \right)^{6}
\right] =\\
&=
4\varepsilon\left[
-\frac{12\sigma}{r_{ij}^{2}}\left( \frac{\sigma}{r_{ij}} \right)^{11} +
\frac{6\sigma}{r_{ij}^{2}}\left( \frac{\sigma}{r_{ij}} \right)^{5}
\right] =\\
&= 
4\varepsilon\left[
-12\left( \frac{\sigma^{12}}{r_{ij}^{13}} \right) +
6\left( \frac{\sigma^{6}}{r_{ij}^{7}} \right)
\right]
\end{align*}
$$

In definitiva:
$$
-\frac12 \sum_{i\neq j} \frac{\partial\phi(r_{ij})}{\partial x_k} = 
-\sum_{j\neq k}\ '\ 2\varepsilon \frac{(x_k-x_j)}{r_{kj}} \left[
-12\left( \frac{\sigma^{12}}{r_{kj}^{13}} \right) +
6\left( \frac{\sigma^{6}}{r_{kj}^{7}} \right)
\right]
$$
Raccolto:
$$
f_k(x) = 24\sigma^6\varepsilon\sum_{i\neq k} \frac{1}{r_{ik}^8}
\left[
\frac{2\sigma^6}{r_{ik}^6}-1
\right]
\left(
x_i-x_k
\right)
$$

#### Derivata di Lennard-Jones

In [7]:
def addendo_derivata_lennard_jones(q_k, q_j, r_kj, sigma=2.644, epsilon=0.345):
    ''' 
    Derivata del potenziale di Lennard-Jones tra due atomi k e j
    rispetto a una coordinata del k-esimo atomo.
    '''
    return 2*epsilon*(q_k-q_j)/r_kj*(-12*(sigma**12/r_kj**13)+6*(sigma**6/r_kj**7))

In [8]:
vec_forza = [] # ciascun entrata è un atomo, ciascun atomo è una lista di tre entrate
for i, atom_neighbours in enumerate(which_neighbour):
    forza_x, forza_y, forza_z = 0, 0, 0
    for j, neighbour_index in enumerate(atom_neighbours):
        d_ij = how_distant[i][j]
        forza_x += addendo_derivata_lennard_jones(vec_x[i], vec_x[neighbour_index], d_ij)
        forza_y += addendo_derivata_lennard_jones(vec_y[i], vec_y[neighbour_index], d_ij)
        forza_z += addendo_derivata_lennard_jones(vec_z[i], vec_z[neighbour_index], d_ij)
        
        vec_forza.append([-forza_x, -forza_y, -forza_z])